## AI-Powered Dynamic Inventory Optimization and Alert System

### Dataset Used: M5 Forecasting - Accuracy

#### Files
- calendar.csv - Contains information about the dates on which the products are sold.
- sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
- sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.
- sell_prices.csv - Contains information about the price of the products sold per store and date.
- sales_train_evaluation.csv - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)


In [0]:
%sql
SHOW TABLES IN workspace.database;

In [0]:
calendar_df=spark.read.table("workspace.database.calendar")
sales_train_df=spark.read.table("workspace.database.sales_train_evaluation")
sales_val_df=spark.read.table("workspace.database.sales_train_validation")
sell_prices_df=spark.read.table("workspace.database.sell_prices")


In [0]:
display(sales_val_df.limit(5))

In [0]:
display(sell_prices_df.limit(5))

In [0]:
display(calendar_df.columns)

Importing necessary libraries

In [0]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from functools import reduce

Working on 3 stores 2 categories and 2 store in 1 state for 3 years

### Starting with calendar.csv 

In [0]:
display(calendar_df.select("year").distinct())

In [0]:
#Filtering the data to just 3 years
calendar_df2=calendar_df.filter(col("year").isin([2011,2012,2013]))
display(calendar_df2.select("year").distinct())

In [0]:
#Dropping unnecessary columns
calendar_df2=calendar_df2.drop("snap_TX","snap_WI")
display(calendar_df2.columns)

In [0]:
null_check=calendar_df2.filter(
    reduce(lambda a,b: a|b,[col(c).isNull() for c in calendar_df2.columns])
)
display(calendar_df2.select(["event_type_1","event_name_2","event_type_2"]).distinct())

In [0]:
#Calculating list of days from 2011-2013
Days_2011_2023=[row["d"] for row in calendar_df2.select("d").collect()]
print(f"Number of days:{len(Days_2011_2023)}")

In [0]:
display(calendar_df2.limit(5))

###Working with sell_prices.csv

In [0]:
sell_prices_df.columns


In [0]:
display(sell_prices_df.select("store_id").distinct())

In [0]:
#Working with only 3 stores in california
sell_prices_df2=sell_prices_df.filter(col("store_id").isin(["CA_1","CA_2","CA_3"]))
display(sell_prices_df2.select("store_id").distinct())

In [0]:
#Checking for the categories now
display(sell_prices_df2.select("item_id").distinct())

In [0]:
#Now again breaking into only 2 categories
sell_prices_df2=sell_prices_df2.filter(~col("item_id").like("FOODS%"))
display(sell_prices_df2.limit(10))

In [0]:
len(sales_train_df.columns)


In [0]:
display(sales_train_df.select('id',
 'item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id'))

In [0]:
display(sales_train_df.select("state_id").distinct())

In [0]:
sales_train_df2=sales_train_df.filter(col("store_id").isin(["CA_1","CA_2","CA_3"]))
display(sales_train_df2.select("store_id").distinct())

In [0]:
sales_train_df2=sales_train_df2.filter(~col("cat_id").isin("FOODS"))
display(sales_train_df2.select("cat_id").distinct())


In [0]:
display(sales_train_df2.select("cat_id").distinct())

In [0]:
display(sales_val_df.limit(20))

In [0]:
display(sales_val_df.select("store_id").distinct())

In [0]:
sales_val_df2=sales_val_df.filter(col("store_id").isin(["CA_1","CA_2","CA_3"]))
display(sales_val_df2.select("store_id").distinct())

In [0]:
sales_val_df2=sales_val_df2.filter(~col("cat_id").isin("FOODS"))
display(sales_val_df2.select("cat_id").distinct())

In [0]:
display(sales_train_df2.select("cat_id","store_id","state_id").distinct())

In [0]:
#Filtering the days to just 3 years
all_columns=sales_train_df2.columns

#Days of 3 years
days_to_keep=[f"d_{i}" for i in range(1,1069)]

#ID columns
id_columns=["id","item_id","dept_id","cat_id","store_id","state_id"]

#Filtering the days
columns_to_select=id_columns+days_to_keep
sales_train_filtered=sales_train_df2.select(columns_to_select)
display(len(sales_train_filtered.columns))

In [0]:
#Melting the columns into rows
#Creating stack expression

stack_expression=", ".join([f"'{day}',{day}" for day in days_to_keep])

sales_train_filtered2=sales_train_filtered.selectExpr(
    *id_columns,
    f"stack({len(days_to_keep)},{stack_expression}) as (day_col,sales)"
)
display(sales_train_filtered2)

In [0]:
display(sales_train_filtered2.select("day_col").distinct().count())

In [0]:
merged_df=sales_train_filtered2.join(
    calendar_df2.select('d','date','wm_yr_wk','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA'),
    sales_train_filtered2.day_col==calendar_df2.d,
    how='left'
).drop("d")


In [0]:
display(merged_df.limit(5))

In [0]:
#Final dataset
daily_sales_df=merged_df.join(
    sell_prices_df2.select("store_id","item_id","wm_yr_wk","sell_price"),
    on=["store_id","item_id","wm_yr_wk"],
    how='left'
)
display(daily_sales_df.limit(10))

In [0]:
daily_sales_df.dtypes

In [0]:
daily_sales_df.printSchema()

In [0]:
#Checking for null values
daily_sales_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in daily_sales_df.columns]).display()


In [0]:
#Checking for null values
daily_sales_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in daily_sales_df.columns]).display()

In [0]:
#Handling missing values for event columns

daily_sales_df=(
    daily_sales_df
    .withColumn("event_name_1",F.when(F.col("event_name_1").isNull(),"No event").otherwise(F.col("event_name_1")))
    .withColumn("event_type_1",F.when(F.col("event_type_1").isNull(),"None").otherwise(F.col("event_type_1")))
    .withColumn("event_name_2",F.when(F.col("event_name_2").isNull(),"No event").otherwise(F.col("event_name_2")))
    .withColumn("event_type_2",F.when(F.col("event_type_2").isNull(),"None").otherwise(F.col("event_type_2")))
)

In [0]:
#Creating a binary has_event flag for analysis
daily_sales_df=daily_sales_df.withColumn(
    "has_event", 
    F.when(
        (F.col("event_name_1")!="No event")|(F.col("event_name_2")!="No event"),
        F.lit(1)
        ).otherwise(F.lit(0))
)

In [0]:
daily_sales_df.groupBy("has_event").count().show()

In [0]:
total_missing=daily_sales_df.filter(F.col("sell_price").isNull()).count()
total_rows=daily_sales_df.count()
display({"Total Missing value percentage":(total_missing/total_rows)*100})

In [0]:
#Analyzing the pattern in missing values because sell_price is one of the important feature
display(
    daily_sales_df
    .filter(F.col("sell_price").
            isNull())
    .groupBy("store_id","cat_id")
    .agg(F.count("*").alias("missing count"))
    .orderBy(F.desc("missing count"))
)

###Missing Value Trends

In [0]:
daily_sales_df=daily_sales_df.withColumn(
    "missing_prices",
    F.when(F.col("sell_price").isNull(), 1).otherwise(0)
)

missing_by_time = (
    daily_sales_df
    .groupBy("date")
    .agg(F.mean("missing_prices").alias("missing_price_mean"))
)
display(missing_by_time)

In [0]:
import seaborn as sns 
import matplotlib.pyplot as plt
#Converting Spark Dataframe to pandas
missing_by_pd= (missing_by_time.toPandas().sort_values(by='date'))

#Plot
plt.figure(figsize=(10,5))
sns.lineplot(data=missing_by_pd,
             x='date',
             y='missing_price_mean',
             color='mediumslateblue')
plt.title('Trend of missing sell prices over time')
plt.xlabel('Date')
plt.ylabel('Proportion of missing sell prices')
plt.grid(True,alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
#Aggregate missingness by store and month for visualization 
missing_by_store_month=(
    daily_sales_df
    .withColumn("year_month",F.date_format("date","yyyy-MM"))
    .groupBy("store_id","year_month")
    .agg(F.mean("missing_prices").alias("missing_prices"))
    .orderBy("store_id","year_month")
)

missing_by_store_month_pd=missing_by_store_month.toPandas()
missing_pivot=missing_by_store_month_pd.pivot(
    index="store_id",columns="year_month",values="missing_prices"
)

plt.figure(figsize=(10,5))
sns.heatmap(missing_pivot,cmap="Purples",linewidths=0.5,annot=False)
plt.title("Missing Sell Price Rate by Store and Month")
plt.xlabel("Month")
plt.ylabel("Store")
plt.xticks(rotation=45,ha="right")
plt.tight_layout()
plt.show()

For all 3 stores, the missing sell price rate is consistently high at the beginning of the period, over time the rate gradually decreases for all the stores. 
It clearly shows the positive trend, now exploring the cause of it.

In [0]:
#Analyzing missing pattern by time

missing_by_week=(
    daily_sales_df
    .groupBy("wm_yr_wk")
    .agg(F.mean("missing_prices").alias("missing_price_rate"))
    .orderBy("wm_yr_wk")
)
missing_by_week_pd=missing_by_week.toPandas()
plt.figure(figsize=(10,5))
sns.lineplot(data=missing_by_week_pd,
             x='wm_yr_wk',
             y='missing_price_rate',
             color='mediumslateblue')
plt.title('Trend of missing sell prices over time')
plt.xlabel('Week')
plt.ylabel('Proportion of missing sell prices')
plt.grid(True,alpha=0.3)    
plt.tight_layout()
plt.show()

In [0]:
#Analyzing missing value by store
missing_by_store=(
    daily_sales_df
    .groupBy("store_id")
    .agg(F.mean("missing_prices").alias("missing_price_rate"))
)
missing_by_store_pd=missing_by_store.toPandas()
plt.figure(figsize=(10,5))
sns.barplot(data=missing_by_store_pd,
            x='store_id',
            y='missing_price_rate',
            color='mediumslateblue')

plt.title('Missing sell price rate by store',fontsize=14,weight='bold')
plt.xlabel('Store')
plt.ylabel('Proportion of missing sell prices')
plt.grid(True,alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
#Aanalyzing missing value by category
missing_by_cat=(
    daily_sales_df
    .groupBy("cat_id")
    .agg(F.mean("missing_prices").alias("missing_price_rate"))
)
missing_by_cat_pd=missing_by_cat.toPandas()
plt.figure(figsize=(10,5))
sns.barplot(data=missing_by_cat_pd,
            x='cat_id',
            y='missing_price_rate',
            color='mediumslateblue')
plt.title('Missing sell price rate by category')
plt.xlabel('Category')
plt.ylabel('Proportion of missing sell prices')
plt.grid(True,alpha=0.3)
plt.tight_layout()
plt.show()

The proportion of missing sell_prices values remains consistent across all stores and product categories.

#### Variability of Missingness

In [0]:
#Checking the variability of missingness
summary_stats=(
    daily_sales_df.
    groupBy("store_id","cat_id").
    agg(F.mean("missing_prices").alias("missing_rate"))
    .agg(
        F.mean("missing_rate").alias("mean"),
        F.stddev("missing_rate").alias("std")
    )
    .toPandas()
)
print(summary_stats)

We can conclude from the results that missingness is consistent as variability is really low.

Also the earlier years has more missing data than later years so might be improvement in data collection or some items may have entered the market later or
sell_prices and sales_train were merged on wm_yr_wk, early weeks might have fewer matches.

In [0]:
#checking if missing prices occur mainly when sales=0

missing_vs_sales=(
    daily_sales_df
    .groupBy("missing_prices")
    .agg(F.mean("sales").alias("mean_sales"))
    .toPandas()
) 
print(missing_vs_sales)

missing prices=1 mean _sales=0
when price is missing, there is no sales
missing prices=0 mean _sales=1.18
when price is available, there is actual sales

Shows strong correlation between missing prices and zero sales, so missingness is not random. That indicates MNAR(Missing Not at Random)

In [0]:
#Checking if the missingness is due to the join issue
total_missing_df=daily_sales_df.filter(F.col("missing_prices")==1)
total_missing=total_missing_df.count()

#Checking if these rows exist in sell_prices_df2 or no
matches=total_missing_df.join(sell_prices_df2,
                              on=['store_id','item_id','wm_yr_wk'],
                              how='inner'
                             ).count()
print(f"Missing prices could have matched(join issues):{matches}")
print(f"Missing prices actually absent in sales df:{total_missing-matches}")

####Since its not join issues so we will perform imputation on it
We will use a predictive model for missing values

####Train and Test Split

In [0]:
#Creating test and train dataset

not_missing_df=daily_sales_df.filter(F.col("missing_prices")==0)
missing_df=daily_sales_df.filter(F.col("missing_prices")==1)

print("Non missing prices:",not_missing_df.count())
print("Missing prices:",missing_df.count())

#Splitting
train_df,val_df=not_missing_df.randomSplit([0.8,0.2],seed=42)
test_df=missing_df
print("Train Count:",train_df.count())
print("Validation Count:",val_df.count())
print("Test Count:",test_df.count())

####Using Gradient Boost for predictive analysis

In [0]:

daily_sales_df.describe(["sales","sell_price"]).display()

In [0]:
#Category Level Analysis
daily_sales_df.groupBy("cat_id").agg(F.sum("sales").alias("total_sales")).display()

In [0]:
#Store and State-level Analysis
daily_sales_df.groupBy("store_id").agg(F.sum("sales").alias("total_sales")).display()